<a href="https://colab.research.google.com/github/qreynas/sentiment_analysis/blob/main/01_Sentiment_Analysis_BCXP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Cleaning

Bootcap Xperience. Este ejercicio esta basado en Sentiment140 Dataset.

El conjunto de datos es una colección de 1,6 millones de tuits que **han sido etiquetados como positivos o negativos**.


Pregunta de trabajo: ¿cómo consiguieron tantos tuits etiquetados? Si buscas en su documentación, [la respuesta está escondida aquí](http://help.sentiment140.com/for-students):

> En nuestro enfoque, asumimos que cualquier tuit con emoticones positivos, como :), era positivo, y los tuits con emoticones negativos, como :(, eran negativos.

Es bueno discutir eso más adelante, pero por ahora, simplemente limpiemos el asunto. En este cuaderno, eliminaremos las columnas que no queremos y estandarizaremos la columna de sentimiento.



#Prep work

In [6]:
import pandas as pd

In [11]:
#Get data. Conect to goole drive
df = pd.read_csv('/content/drive/MyDrive/2025/02 BCXP ML/01 Análisis de sentimientos/training_1600000_processed_noemoticon.csv', encoding='latin-1')

In [13]:
df.head(3)

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599999 entries, 0 to 1599998
Data columns (total 6 columns):
 #   Column                                                                                                               Non-Null Count    Dtype 
---  ------                                                                                                               --------------    ----- 
 0   0                                                                                                                    1599999 non-null  int64 
 1   1467810369                                                                                                           1599999 non-null  int64 
 2   Mon Apr 06 22:19:45 PDT 2009                                                                                         1599999 non-null  object
 3   NO_QUERY                                                                                                             1599999 non-null  object
 4   _